
# Tutorials :- How to implement WebSockets in Angular Project



##  How to implement WebSockets in Angular Project

* In this tutorial we will learn how to implement a service to interact through WebSocket API.
The WebSocket API is an advanced technology that allows communication between the client's browser and the server, both to send messages and to receive responses without the need to make a new request to the server to get the answer.



##  WebSocket Flow

* To make a WebSocket connection 
  * it is necessary to use the HTTP protocol upgrade mechanism, 
  * this allows us to move from an HTTP 1.1 connection to HTTP 2.0 (which allows using server push feature),
  *  or as in this case to move from an HTTP 1.1 connection to a WebSocket connection.
* The client makes the request using HTTP methods such as GET or POST, 
  * the server evaluates the request and 
  * if the response is positive the server returns a status code 101 
    * with the headers "Connection" and "Upgrade". 
* After that, 
  * as the connection between client and server exists, 
  * it is possible to exchange messages and event-based responses 
  * until one of the two sides closes the connection.




## The following image describes the flow explained in previous paragraphs.

![flow of web Socket](./image/0013.Angular.0002.websocket/flow_of_webSocket.png)

## WebSocket Server

* It’s a TCP application that can be written in any server-side programming language. 
* Its function will be to manage connections, sending and receiving messages. 
* It’s recommended to separate these servers from other applications and use a reverse proxy.

* You can use this "Node + Express" WebSocket Server for angular client implementation: https://github.com/davrv93/ws-server.

## Implementation

* We will create an angular project using the command ng new, 
  * in this case the name of the project will be socketrv, 
  * likewise we create websocket.service.ts inside a folder called services.

![create_websocket_service_file](./image/0013.Angular.0002.websocket/create_websocket_service_file.png)

## WebSocketService

* We will create a service called WebSocketService which will allow us to connect to the WebSocket server. 
* WebSocketService class has following overall structure: A list with bullet points:



##### Properties 
* a) subject: Is an instance of AnonymousSubject class with MessageEvent property. The AnonymousSubject class allows extending a Subject by defining the source and destination observables. 
* b) message: Is an instance of Subject class. Every Subject is an Observable and an Observer. We'll subscribe to this Subject, and we'll be able to call next to feed values as well as error and complete.



##### Methods 
* a) connect: This method validates if subject property doesn’t exist and then calls create method for creating the subject. 
* b) create: This method creates the AnonymousSubject that will be used for subscriptions.



##### Interfaces 
  * a) Message: It is a dict that defines the behavior of an object, but does not specify its content, the attributes will be source and content, both of type string.


### websocket  Constructor

##### WebSocket()
Returns a newly created WebSocket object.



### websocket Instance properties
##### WebSocket.binaryType
The binary data type used by the connection.

##### WebSocket.bufferedAmount Read only
The number of bytes of queued data.

##### WebSocket.extensions Read only
The extensions selected by the server.

##### WebSocket.protocol Read only
The sub-protocol selected by the server.

##### WebSocket.readyState Read only
The current state of the connection.

##### WebSocket.url Read only
The absolute URL of the WebSocket.



### websocket Instance properties :- 
##### onclose :-  
* Fired when a connection with a WebSocket is closed. 
* Also available via the close event

##### onerror :-  
* Fired when a connection with a WebSocket has been closed because of an error, 
  * such as when some data couldn't be sent. 
* Also available via the error event.

##### onmessage
* Fired when data is received through a WebSocket. 
* Also available via the message event.

##### onopen
Fired when a connection with a WebSocket is opened. 
Also available via the open event.



### websocket Instance methods
##### WebSocket.close()
Closes the connection.

##### WebSocket.send()
Enqueues data to be transmitted.



### websocket Events
* Listen to these events using addEventListener() or by assigning an event listener to the oneventname property of this interface.

##### close
* Fired when a connection with a WebSocket is closed. 
* Also available via the onclose property

##### error
* Fired when a connection with a WebSocket has been closed because of an error, 
  * such as when some data couldn't be sent. 
* Also available via the onerror property.

##### message
* Fired when data is received through a WebSocket. 
* Also available via the onmessage property.

##### open
* Fired when a connection with a WebSocket is opened. 
* Also available via the onopen property.


##### And the code for the same is given below:

```ts
// src\app\services\websocket.service.ts
import { Injectable } from "@angular/core";
import { Observable, Observer } from 'rxjs';
import { AnonymousSubject } from 'rxjs/internal/Subject';
import { Subject } from 'rxjs';
import { map } from 'rxjs/operators';

const CHAT_URL = "ws://localhost:5000";

export interface Message {
    source: string;
    content: string;
}

@Injectable()
export class WebsocketService {
    private subject: AnonymousSubject<MessageEvent>;
    public messages: Subject<Message>;

    constructor() {
        this.messages = <Subject<Message>>this.connect(CHAT_URL).pipe(
            map(
                (response: MessageEvent): Message => {
                    console.log(response.data);
                    let data = JSON.parse(response.data)
                    return data;
                }
            )
        );
    }

    public connect(url): AnonymousSubject<MessageEvent> {
        if (!this.subject) {
            this.subject = this.create(url);
            console.log("Successfully connected: " + url);
        }
        return this.subject;
    }

    private create(url): AnonymousSubject<MessageEvent> {
        let ws = new WebSocket(url);
        let observable = new Observable((obs: Observer<MessageEvent>) => {
            ws.onmessage = obs.next.bind(obs);
            ws.onerror = obs.error.bind(obs);
            ws.onclose = obs.complete.bind(obs);
            return ws.close.bind(ws);
        });
        let observer = {
            error: null,
            complete: null,
            next: (data: Object) => {
                console.log('Message sent to websocket: ', data);
                if (ws.readyState === WebSocket.OPEN) {
                    ws.send(JSON.stringify(data));
                }
            }
        };
        return new AnonymousSubject<MessageEvent>(observer, observable);
    }
}
```
